# CNN architecture


In this notebook, we aim to evaluate the performance of the Convolutional Autoencoder architecture.

In [ ]:
import sys
import importlib
import pickle

# data manipulation
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# data analysis
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA

# pytorch specific
import torch
import torch.optim as optim
import torch.nn.functional as F
from tqdm import tqdm

# project specific
sys.path.append('../')
from src import config
from src.utils import experiment, helpers



from matplotlib.animation import FuncAnimation
from IPython.display import HTML

pd.options.display.width = 1000
DEVICE = torch.device(config["DEVICE"])

## Grid

Here we define the parametters to search through

In [ ]:
# We first set the parametters that we do no want to change

data_params = { 
    "LS_threshold" : 0.0023,
    "MAD_threshold" : 1, 
    "MT_removal" : True, 
    "expression_threshold" : 0.1}

# or we can use a pre loaded dataset to save time
data_params = '../workfiles/light_BRCA_ds.pkl'

model_params = {
    "dropout" : 0.2,
    "latent_dim" : 64,
    "variational" : False,
    "convolution": True,
    "transformer" : False
    }



In [ ]:
importlib.reload(helpers)


dynamic_params = {
    "dropout" : [0.1, 0.3, 0.5],
    "latent_dim": [16, 32, 64, 128],
    # ... (any other uncoupled parameters)
    # Coupled parameters are passed as tuples within the list
    "conv_params": [
        (("padding", 3), ("kernel_size", 7)),
        (("padding", 2), ("kernel_size", 5)),
        (("padding", 1), ("kernel_size", 3))
    ]
    # Note: The key "conv_params" is only a placeholder and won't appear in the final configs.
}

configurations = helpers.generate_config(model_params, dynamic_params)


In [ ]:
print(configurations[0])
print(len(configurations))

In [ ]:
importlib.reload(experiment)
experiments = []

for config in configurations:
    e = experiment.Experiment(data_param=data_params, model_param=config, n_epoch= 3000)
    e.run()

    experiment_data = {**config,'score': e.metric}
    experiments.append(experiment_data)

In [ ]:
results = pd.DataFrame(experiments)
print(results)

In [ ]:
results.to_csv("../workfiles/CNN_results.csv")

In [ ]:
# Find the experiment with the highest score.
best_experiment = max(experiments, key=lambda exp: exp['score'])

# 'best_experiment' now holds the dictionary with the highest 'score' value.
best_config = best_experiment
highest_score = best_experiment['score']

# You can print or otherwise use 'best_config' and 'highest_score' as needed.
print("Best configuration:", best_config)
print("Highest score:", highest_score)